In [1]:
import re
import math
import numpy as np
from itertools import chain
from collections import Counter
import nltk
from nltk.util import ngrams # This is the ngram magic.
from music21 import *
from pathlib import Path 
import pandas as pd
from os import listdir
from os.path import isfile, join
from itertools import groupby
import glob
from difflib import SequenceMatcher
from timeit import default_timer as timer
from datetime import timedelta
us = environment.UserSettings()
us['musicxmlPath'] = 'C:\\Program Files\\MuseScore 3\\bin\\MuseScore3.exe'
us['musescoreDirectPNGPath'] = 'C:\\Program Files\\MuseScore 3\\bin\\MuseScore3.exe'
us['musicxmlPath']

WindowsPath('C:/Program Files/MuseScore 3/bin/MuseScore3.exe')

In [43]:
df = pd.read_csv(r'bach_analysis_15.csv')

df

c:\users\lleis\appdata\local\programs\python\python39\lib\site-packages\IPython\core\interactiveshell.py:3441: DtypeWarning: Columns (2,4) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


,Unnamed: 0,Notes,LCS,lenght,LARNOS,rests,chords,consider_measure_idx,consider_note_duration,offset_notes,repetition
0,64872,4_3 + 230_2,"[(44, 23, ('67', '65', '64', '69', '69', '68',...",8.0,0.0,False,True,No,No,True,True
1,64889,4_3 + 230_2,"[(44, 23, ('67', '65', '64', '69', '69', '68',...",8.0,0.0,False,False,No,No,True,False
2,13401,4_3 + 47_2,"[(48, 11, ('69', '68', '69', '67', '65', '64',...",8.0,0.0,True,False,No,No,True,False
3,13385,4_3 + 47_2,"[(48, 11, ('69', '68', '69', '67', '65', '64',...",8.0,0.0,True,True,No,No,True,False
4,13416,4_3 + 47_2,"[(48, 11, ('69', '68', '69', '67', '65', '64',...",8.0,0.0,False,True,No,No,True,True
...,...,...,...,...,...,...,...,...,...,...,...
112250,40036,4_3 + 140_1,0.0,0.0,0.0,False,True,Yes,Yes,True,True
112251,40035,4_3 + 140_1,0.0,0.0,0.0,False,True,Yes,No,False,False
112252,40034,4_3 + 140_1,0.0,0.0,0.0,False,True,Yes,No,False,True
112253,40033,4_3 + 140_1,0.0,0.0,0.0,False,True,Yes,No,True,False


In [46]:
data = df.loc[(df["lenght"] > 0)]
results = data.groupby(['rests', 'chords', 'consider_measure_idx', 'consider_note_duration', 'offset_notes', 'repetition']).size().sort_values(ascending=False).reset_index(name='count')
results

,rests,chords,consider_measure_idx,consider_note_duration,offset_notes,repetition,count
0,False,True,No,No,True,False,727
1,False,False,No,No,True,False,727
2,False,False,No,No,True,True,727
3,False,True,No,No,True,True,727
4,True,False,No,No,True,False,720
...,...,...,...,...,...,...,...
59,False,True,Yes,Yes,False,False,6
60,False,False,Yes,Yes,False,True,6
61,True,True,Yes,Yes,False,False,6
62,True,True,Yes,Yes,False,True,6


In [47]:
data = df.loc[(df["lenght"] == 0)]
results = data.groupby(['rests', 'chords', 'consider_measure_idx', 'consider_note_duration', 'offset_notes', 'repetition']).size().sort_values(ascending=False).reset_index(name='count')
results

,rests,chords,consider_measure_idx,consider_note_duration,offset_notes,repetition,count
0,True,True,Yes,Yes,False,True,1748
1,True,True,Yes,Yes,False,False,1748
2,True,False,Yes,Yes,False,True,1748
3,True,False,Yes,Yes,False,False,1748
4,False,True,Yes,Yes,False,True,1748
...,...,...,...,...,...,...,...
59,True,False,No,No,True,False,1034
60,False,True,No,No,True,True,1027
61,False,True,No,No,True,False,1027
62,False,False,No,No,True,True,1027


In [21]:
data['lenght'].value_counts()

4.0    9304
5.0    2968
6.0    1060
7.0     208
8.0      16
Name: lenght, dtype: int64

In [22]:
results = data.groupby(['rests', 'chords', 'consider_measure_idx', 'consider_note_duration', 'offset_notes', 'repetition']).size().sort_values(ascending=False)[:20] 

In [23]:
print(type(results))

<class 'pandas.core.series.Series'>


In [27]:
res = results.to_frame()

In [17]:
results.to_csv('bach_results20_2.csv')

In [29]:
dataframe = pd.DataFrame() 

dataframe = dataframe.append(res)

dataframe


0
rests chords consider_measure_idx consider_note_duration offset_notes repetition     
False True   No                   No                     True         False       727
      False  No                   No                     True         False       727
                                                                      True        727
      True   No                   No                     True         True        727
True  False  No                   No                     True         False       720
      True   No                   No                     True         False       720
                                                                      True        720
      False  No                   No                     True         True        720
False False  No                   No                     False        False       431
True  True   No                   No                     False        False       431
                                                                      True        431
      False  No                   No                     False        False       431
                                                                      True        431
False True   No                   No                     False        False       431
      False  No                   No                     False        True        431
      True   No                   No                     False        True        431
                                  Yes                    True         False       269
                                                                      True        269
      False  No                   Yes                    True         False       269
                                                                      True        269

In [40]:
dataframe = pd.DataFrame() 
for i in range(10, 100, 5):
    df = pd.read_csv(r'bach_analysis_'+ str(i) + '.csv')
    #data = df.loc[(df["lenght"] > 0)]
    results = df.groupby(['rests', 'chords', 'consider_measure_idx', 'consider_note_duration', 'offset_notes', 'repetition']).size().sort_values(ascending=False).reset_index(name='count') 

    #res = results.to_frame()
    dataframe = dataframe.append(res)
    print(i)
    
dataframe
    


c:\users\lleis\appdata\local\programs\python\python39\lib\site-packages\IPython\core\interactiveshell.py:3441: DtypeWarning: Columns (2,4) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


10
15
20
25
30
35
40
45
50
55
60
65
70
75
80
85
90
95


0
rests chords consider_measure_idx consider_note_duration offset_notes repetition     
False True   No                   No                     True         True        902
                                                                      False       902
      False  No                   No                     True         False       902
                                                                      True        902
True  False  No                   No                     True         False       897
...                                                                               ...
      True   No                   No                     False        False       294
      False  No                   Yes                    True         True        288
                                                                      False       288
False True   No                   Yes                    True         True        288
                                                                      False       288

[360 rows x 1 columns]

In [41]:
dataframe

0
rests chords consider_measure_idx consider_note_duration offset_notes repetition     
False True   No                   No                     True         True        902
                                                                      False       902
      False  No                   No                     True         False       902
                                                                      True        902
True  False  No                   No                     True         False       897
...                                                                               ...
      True   No                   No                     False        False       294
      False  No                   Yes                    True         True        288
                                                                      False       288
False True   No                   Yes                    True         True        288
                                                                      False       288

[360 rows x 1 columns]

## GROUPING ALL THE RESULTS TOGETHER

In [11]:
li = []
for i in range(1, 100, 1):
    df = pd.read_csv(r'bach_analysis_'+ str(i) + '.csv', index_col=None, header=0)
    li.append(df)

frame = pd.concat(li, axis=0, ignore_index=True)

frame

c:\users\lleis\appdata\local\programs\python\python39\lib\site-packages\IPython\core\interactiveshell.py:3441: DtypeWarning: Columns (2,4) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


,Unnamed: 0,Notes,LCS,lenght,LARNOS,rests,chords,consider_measure_idx,consider_note_duration,offset_notes,repetition
0,18842,1_2 + 67_2,"[(23, 0, ('60', '67', '64', '60', '67', '69', ...",34.0,"[(59, 36, ('64', '62', '60', '72', '71', '69',...",True,False,No,No,False,True
1,18859,1_2 + 67_2,"[(23, 0, ('60', '67', '64', '60', '67', '69', ...",34.0,"[(59, 36, ('64', '62', '60', '72', '71', '69',...",False,True,No,No,False,False
2,18875,1_2 + 67_2,"[(23, 0, ('60', '67', '64', '60', '67', '69', ...",34.0,"[(59, 36, ('64', '62', '60', '72', '71', '69',...",False,False,No,No,False,False
3,18874,1_2 + 67_2,"[(23, 0, ('60', '67', '64', '60', '67', '69', ...",34.0,"[(59, 36, ('64', '62', '60', '72', '71', '69',...",False,False,No,No,False,True
4,73496,1_2 + 264_1,"[(37, 19, ('69', '67', '64', '69', '67', '65',...",34.0,"[(6, 9, ('69', '67', '67', '69', '71', '72', '...",True,False,No,No,True,True
...,...,...,...,...,...,...,...,...,...,...,...
11113240,39885,23_3 + 139_4,0.0,0.0,0.0,True,True,No,Yes,True,False
11113241,39884,23_3 + 139_4,0.0,0.0,0.0,True,True,No,Yes,True,True
11113242,39883,23_3 + 139_4,0.0,0.0,0.0,True,True,No,No,False,False
11113243,39882,23_3 + 139_4,0.0,0.0,0.0,True,True,No,No,False,True


In [42]:
results = frame.groupby(['rests', 'chords', 'consider_measure_idx', 'consider_note_duration', 'offset_notes', 'repetition']).size().sort_values(ascending=False).reset_index(name='count')
results

,rests,chords,consider_measure_idx,consider_note_duration,offset_notes,repetition,count
0,False,True,Yes,Yes,True,True,173646
1,False,False,No,No,False,False,173646
2,True,False,No,No,False,True,173646
3,True,False,No,No,True,False,173646
4,True,False,No,No,True,True,173646
...,...,...,...,...,...,...,...
59,False,True,Yes,No,False,True,173646
60,False,True,Yes,No,True,False,173646
61,False,True,Yes,No,True,True,173646
62,False,True,Yes,Yes,False,False,173646


In [43]:
data = frame.loc[(frame["lenght"] > 0)]
data

,Unnamed: 0,Notes,LCS,lenght,LARNOS,rests,chords,consider_measure_idx,consider_note_duration,offset_notes,repetition
0,18842,1_2 + 67_2,"[(23, 0, ('60', '67', '64', '60', '67', '69', ...",34.0,"[(59, 36, ('64', '62', '60', '72', '71', '69',...",True,False,No,No,False,True
1,18859,1_2 + 67_2,"[(23, 0, ('60', '67', '64', '60', '67', '69', ...",34.0,"[(59, 36, ('64', '62', '60', '72', '71', '69',...",False,True,No,No,False,False
2,18875,1_2 + 67_2,"[(23, 0, ('60', '67', '64', '60', '67', '69', ...",34.0,"[(59, 36, ('64', '62', '60', '72', '71', '69',...",False,False,No,No,False,False
3,18874,1_2 + 67_2,"[(23, 0, ('60', '67', '64', '60', '67', '69', ...",34.0,"[(59, 36, ('64', '62', '60', '72', '71', '69',...",False,False,No,No,False,True
4,73496,1_2 + 264_1,"[(37, 19, ('69', '67', '64', '69', '67', '65',...",34.0,"[(6, 9, ('69', '67', '67', '69', '71', '72', '...",True,False,No,No,True,True
...,...,...,...,...,...,...,...,...,...,...,...
11014501,51176,23_3 + 178_1,"[(25, 43, ('65', '64', '62', '64'), ('65', '64...",4.0,0.0,False,True,No,No,True,True
11014502,4889,23_3 + 19_1,"[(34, 58, ('64', '62', '60', '59'), ('64', '62...",4.0,0.0,True,False,No,No,True,False
11014503,102649,23_3 + 375_2,"[(10, 13, ('64', '69', '69', '67'), ('64', '69...",4.0,0.0,False,False,No,No,True,False
11014504,111677,23_3 + 407_2,"[(32, 36, ('59 1.0', '64 1.0', '64 1.0', '62 1...",4.0,0.0,False,False,No,Yes,True,False


In [44]:
results2 = data.groupby(['rests', 'chords', 'consider_measure_idx', 'consider_note_duration', 'offset_notes', 'repetition']).size().sort_values(ascending=False).reset_index(name='count')
results2

,rests,chords,consider_measure_idx,consider_note_duration,offset_notes,repetition,count
0,False,False,No,No,True,False,61565
1,False,False,No,No,True,True,61565
2,False,True,No,No,True,True,61542
3,False,True,No,No,True,False,61542
4,True,False,No,No,True,False,61013
...,...,...,...,...,...,...,...
59,False,True,Yes,Yes,False,False,733
60,False,False,Yes,Yes,False,True,733
61,True,True,Yes,Yes,False,False,733
62,True,True,Yes,Yes,False,True,733


In [45]:
ids = []
for i, row in results.iterrows():
    #print(i)
    id_lst = ''
    for r in range(6):
        
        id_lst+=row.values[r][0]
    ids.append(id_lst)
    

results['method_id'] = ids
results

,rests,chords,consider_measure_idx,consider_note_duration,offset_notes,repetition,count,method_id
0,False,True,Yes,Yes,True,True,173646,FTYYTT
1,False,False,No,No,False,False,173646,FFNNFF
2,True,False,No,No,False,True,173646,TFNNFT
3,True,False,No,No,True,False,173646,TFNNTF
4,True,False,No,No,True,True,173646,TFNNTT
...,...,...,...,...,...,...,...,...
59,False,True,Yes,No,False,True,173646,FTYNFT
60,False,True,Yes,No,True,False,173646,FTYNTF
61,False,True,Yes,No,True,True,173646,FTYNTT
62,False,True,Yes,Yes,False,False,173646,FTYYFF


In [46]:
ids = []
for i, row in results2.iterrows():
    #print(i)
    id_lst = ''
    for r in range(6):
        
        id_lst+=row.values[r][0]
    ids.append(id_lst)
    

results2['method_id'] = ids
results2

,rests,chords,consider_measure_idx,consider_note_duration,offset_notes,repetition,count,method_id
0,False,False,No,No,True,False,61565,FFNNTF
1,False,False,No,No,True,True,61565,FFNNTT
2,False,True,No,No,True,True,61542,FTNNTT
3,False,True,No,No,True,False,61542,FTNNTF
4,True,False,No,No,True,False,61013,TFNNTF
...,...,...,...,...,...,...,...,...
59,False,True,Yes,Yes,False,False,733,FTYYFF
60,False,False,Yes,Yes,False,True,733,FFYYFT
61,True,True,Yes,Yes,False,False,733,TTYYFF
62,True,True,Yes,Yes,False,True,733,TTYYFT


In [47]:
df = pd.merge(results,results2[['method_id','count']],on='method_id', how='left')
df

,rests,chords,consider_measure_idx,consider_note_duration,offset_notes,repetition,count_x,method_id,count_y
0,False,True,Yes,Yes,True,True,173646,FTYYTT,1453
1,False,False,No,No,False,False,173646,FFNNFF,32754
2,True,False,No,No,False,True,173646,TFNNFT,32754
3,True,False,No,No,True,False,173646,TFNNTF,61013
4,True,False,No,No,True,True,173646,TFNNTT,61013
...,...,...,...,...,...,...,...,...,...
59,False,True,Yes,No,False,True,173646,FTYNFT,1991
60,False,True,Yes,No,True,False,173646,FTYNTF,6797
61,False,True,Yes,No,True,True,173646,FTYNTT,6797
62,False,True,Yes,Yes,False,False,173646,FTYYFF,733


In [50]:
df.sort_values(by=['count_y'], ascending=False)

TypeError: 'method' object is not iterable

In [53]:
percentage_lst = []
for i, row in df.iterrows():
    percent = round(row.values[8]*100/row.values[6], 2)
    print(percent)
    

0.84
18.86
18.86
35.14
35.14
5.56
5.56
11.12
11.12
1.15
1.15
3.9
3.9
0.42
0.42
0.86
0.86
18.86
18.86
35.13
35.13
5.56
5.56
11.12
11.12
1.15
1.15
3.9
0.42
0.42
0.86
18.86
0.86
0.84
0.84
18.86
35.45
35.45
5.56
5.56
11.13
11.13
1.15
1.15
3.92
3.92
0.42
0.42
0.84
0.42
18.86
18.86
35.44
35.44
5.56
5.56
11.13
11.13
1.15
1.15
3.91
3.91
0.42
3.9
